# Find the uids of the runs in May 05 beamtime

In [1]:
import pandas as pd

from databroker import catalog

In [2]:
list(catalog)

['test_data_in_database',
 'analysis',
 'bt_safN_306132',
 'processed',
 'beamtime_20210930_saf_308066',
 'beamtime_20210930_saf_308066_processed',
 'pdf',
 'saf_307381',
 'xpd']

In [3]:
db = catalog["pdf"]

In [4]:
def print_sample_name(_db):
    print("uid sample")
    for uid in _db:
        run = _db[uid]
        sample = run.metadata["start"].get("sample_name", "")
        status = run.metadata["stop"].get("exit_status", "")
        if status == "success" and sample != "":
            print("{} {}".format(uid, sample))
    return

In [5]:
print_sample_name(db)

uid sample
a3b9be6c-7149-4a1a-8ae9-c88ca4448642 ML99_powder
480732a3-dd53-4d2e-a1a3-c291a49255f8 ML99_premeleted
a1f18994-96c8-4c92-9910-ab52b7c6bfc6 ML99_premeleted
e3fda015-dfe2-4e2a-8c2e-d717ab099766 ML99_premeleted
3df14c06-6809-45fb-95e5-8462192a853e ML99_premeleted
d3f8e498-0f82-4df4-8b7f-5ec2d35b81b7 ML61_crystalline
eb933df1-5abb-46d4-956e-105aaf905b07 ML61_premelted
798ef19d-5933-4118-978d-6513ac34b818 ML61_premelted
07c6d60b-4c24-40ac-a727-8b7d8c4bf4b5 ML99_blank1
f3017542-3761-43ed-aad2-403b385a23d6 ML99_blank2
985ff3ac-2fb1-4265-b996-6cf985eef623 ML61_blank1
43af3b5b-4df1-488f-b45c-ebf3d3787ea2 44
82205d6c-da2a-47bc-bfc0-7fe84e7c0d23 44
87ad4550-cef7-46eb-9349-4c6f574ef943 44
31dc52cf-adf2-40eb-84fa-3afb4a809660 44
dacfdc56-3b74-465e-85bd-1cbc6e9fef45 44
ba41ae51-3ba3-459b-8329-293b4379dc61 44
1ced1e62-1c90-4c31-942c-e809421b538d 44
343706e0-8075-4ab1-a3c9-278c97324a37 44
dd8fedec-5fb7-443b-aec2-ec01f028d827 Ni_calib
c3c218b8-1101-4a54-9eda-c633a4ffbd7a Ni_calib
c6ef7bae-23

# Process the data

In [26]:
from datareduction.reduction import DataProcessConfig, DataProcessor

import pandas as pd
import xarray as xr
from pathlib import PurePath, Path

In [15]:
config = DataProcessConfig()
config.database.name = "pdf"
config.database.image_key = "pe1c_image"
dp = DataProcessor(config)

In [9]:
df = pd.read_csv("./data/uids_20210505.txt", delimiter=" ")
df

,uid,sample
0,a3b9be6c-7149-4a1a-8ae9-c88ca4448642,ML99_powder
1,480732a3-dd53-4d2e-a1a3-c291a49255f8,ML99_premeleted
2,a1f18994-96c8-4c92-9910-ab52b7c6bfc6,ML99_premeleted
3,e3fda015-dfe2-4e2a-8c2e-d717ab099766,ML99_premeleted
4,3df14c06-6809-45fb-95e5-8462192a853e,ML99_premeleted
5,d3f8e498-0f82-4df4-8b7f-5ec2d35b81b7,ML61_crystalline
6,eb933df1-5abb-46d4-956e-105aaf905b07,ML61_premelted
7,798ef19d-5933-4118-978d-6513ac34b818,ML61_premelted
8,07c6d60b-4c24-40ac-a727-8b7d8c4bf4b5,ML99_blank1
9,f3017542-3761-43ed-aad2-403b385a23d6,ML99_blank2


In [16]:
def process_all(dp, df, output_dir):
    _output_dir = PurePath(output_dir)
    for row in df.itertuples():
        filename = "{}_{}".format(row.uid, row.sample)
        filepath = _output_dir.joinpath(filename)
        dp.process(row.uid)
        dp.rc.dataset.to_netcdf(str(filepath))
    return 

In [17]:
output_dir = "./data/temp2/"
process_all(dp, df, output_dir)

In [18]:
del dp, df, output_dir

# Merge data

In [52]:
def merge_data(target_dir, include, exclude):
    
    def gen_ds():
        for f in Path(target_dir).glob("[!.]*"):
            if include in f.name and exclude not in f.name:
                ds = xr.open_dataset(str(f))
                qmax = ds["Q"].values.max()
                ds = ds.interp({"Q": np.arange(0.01, qmax, 0.01)})
                yield ds
            
    return xr.merge(gen_ds())

In [71]:
merge_data("./data/temp2/", "blank", "xxxxxx").to_netcdf("./data/temp_bkg_data.nc")

In [72]:
merge_data("./data/temp2/", "", "blank").to_netcdf("./data/temp_data.nc")

# Get PDF

In [1]:
import xarray as xr

In [2]:
data = xr.open_dataset("./data/temp_data.nc")
bkg_data = xr.open_dataset("./data/temp_bkg_data.nc")

In [3]:
from datareduction.reduction import ReductionConfig, ReductionCalculator

In [4]:
config = ReductionConfig()

In [5]:
config.pdf.rmax = 50.0
config.pdf.composition = "MnMn"
config.pdf.qmaxinst = 17.0
config.pdf.qmax = 14.0

In [6]:
rc = ReductionCalculator(config)

## ML 61

In [69]:
data2 = data.where(
    data["sample_name"].isin(["ML61_premelted", "ML61_crystalline"]),
    drop=True
)

In [70]:
bkg_data2 = bkg_data.isel({"time": 0})

In [71]:
rc.set_dataset(data2)
rc.set_bkg_dataset(bkg_data2)
rc.bkg_subtract()

In [72]:
rc.interact_fq(0)

interactive(children=(FloatSlider(value=0.0, description='qmin', max=5.0, step=0.05), FloatSlider(value=14.0, …

<function datareduction.reduction.ReductionCalculator.interact_fq.<locals>.func(qmin, qmax, qmaxinst, lowessf, qcutoff, endzero)>

In [73]:
rc.get_G()

In [74]:
data2 = rc.dataset.copy()

# ML99

In [75]:
data1 = data.where(
    data["sample_name"].isin(["ML99_premeleted"]),
    drop=True
)

In [76]:
bkg_data1 = bkg_data.isel({"time": 1})

In [77]:
rc.set_dataset(data1)
rc.set_bkg_dataset(bkg_data1)
rc.bkg_subtract()

In [78]:
rc.interact_fq(0)

interactive(children=(FloatSlider(value=0.0, description='qmin', max=5.0, step=0.05), FloatSlider(value=14.0, …

<function datareduction.reduction.ReductionCalculator.interact_fq.<locals>.func(qmin, qmax, qmaxinst, lowessf, qcutoff, endzero)>

In [79]:
rc.get_G()

In [80]:
data1 = rc.dataset.copy()

## ML 99 powder

In [98]:
data3 = data.where(
    data["sample_name"].isin(["ML99_powder"]),
    drop=True
)

In [99]:
bkg_data3 = bkg_data.isel({"time": 1})

In [101]:
rc.set_dataset(data3)
rc.set_bkg_dataset(bkg_data3)
rc.bkg_subtract()

In [103]:
rc.interact_fq()

interactive(children=(FloatSlider(value=0.0, description='qmin', max=5.0, step=0.05), FloatSlider(value=14.0, …

<function datareduction.reduction.ReductionCalculator.interact_fq.<locals>.func(qmin, qmax, qmaxinst, lowessf, qcutoff, endzero)>

In [104]:
rc.get_G()

In [105]:
data3 = rc.dataset.copy()

In [106]:
final_data = xr.merge([data1, data2, data3])

In [107]:
final_data.to_netcdf("./data/data_20210505.nc")